## 2. Verifica che il progetto sia disponibile in `/content/PV_power-forecasting`


In [ ]:
%cd /content
!ls


## 3. Installa librerie per il training


In [ ]:
!pip install -q torch pandas numpy scikit-learn


## 4. Importa i moduli e prepara i DataLoader


In [ ]:
%cd /content/PV_power-forecasting


In [ ]:
from pathlib import Path
import pandas as pd
import torch

from split import split_and_scale
from dataloader import create_dataloader
from gru_model import GRUDirectModel, train_one_epoch, eval_one_epoch, device

DATA_FILE = Path('processed/merged_dataset_final_interpolated.xlsx')
if not DATA_FILE.exists():
    raise FileNotFoundError('Esegui prima il preprocessing/interpolazione.')

df = pd.read_excel(DATA_FILE)
train_scaled, val_scaled, test_scaled, feature_scaler, target_scaler = split_and_scale(df)

TARGET_COL = 'Production_KWh'
future_time_cols = ['hour_sin', 'hour_cos', 'doy_sin', 'doy_cos']
encoder_cols = [c for c in train_scaled.columns if c not in [TARGET_COL, 'pv_date']]

train_dl = create_dataloader(train_scaled, encoder_cols, future_time_cols,
                             input_length=168, forecast_horizon=24,
                             batch_size=32, shuffle=True)
val_dl = create_dataloader(val_scaled, encoder_cols, future_time_cols,
                           input_length=168, forecast_horizon=24,
                           batch_size=32, shuffle=False)
test_dl = create_dataloader(test_scaled, encoder_cols, future_time_cols,
                            input_length=168, forecast_horizon=24,
                            batch_size=1, shuffle=False)

len(train_dl), len(val_dl), len(test_dl)


## 5. Addestra la GRU


In [ ]:
model = GRUDirectModel(
    enc_in_dim=len(encoder_cols),
    future_time_dim=len(future_time_cols),
    hidden_dim=128,
    num_layers=2,
    forecast_horizon=24
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
EPOCHS = 20
for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(model, train_dl, optimizer, device)
    val_loss = eval_one_epoch(model, val_dl, device)
    print(f'Epoch {epoch:02d} | Train {train_loss:.4f} | Val {val_loss:.4f}')


## 6. Crea la previsione finale e salva su CSV


In [ ]:
import numpy as np

model.eval()
all_predictions = []
with torch.no_grad():
    for x_enc, x_future, y in test_dl:
        x_enc, x_future = x_enc.to(device), x_future.to(device)
        pred_scaled = model(x_enc, x_future).cpu().numpy()[0]
        all_predictions.append(pred_scaled)

last_pred_scaled = np.array(all_predictions[-1]).reshape(-1, 1)
last_pred = target_scaler.inverse_transform(last_pred_scaled).flatten()

last_timestamp = df.index[-1]
future_times = pd.date_range(start=last_timestamp + pd.Timedelta(hours=1),
                             periods=24, freq='H')

df_pred = pd.DataFrame({'timestamp': future_times, 'pv_forecast': last_pred})
output_path = Path('predictions_colab.csv')
df_pred.to_csv(output_path, index=False)
print('✓ Salvato', output_path)
df_pred.head()
